In [57]:
import csv
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


#df = pd.read_csv('deneme.csv')
#print(df.to_string())
#print(df.shape)

#Verilerdeki tüm ögeleri bulmak ve her satırı okuyabilmek için csv paketi kullanılacak.
all_items = set() # Tüm ögeler
with open("All_Data.csv") as f:
    reader = csv.reader(f, delimiter=",") # her bir tag virgülle ayrıldıgı için burada belirtiyorum
    for i, line in enumerate(reader):
        all_items.update(line)

# Her bir ögeden veriler içerisinde kaç tane oldugunun sayılması ve listenin buna göre güncellenmesi
counting = list()
with open("All_Data.csv") as f:
    reader = csv.reader(f, delimiter=",")
    for i, line in enumerate(reader):
        row = {item:0 for item in all_items}
        row.update({item:1 for item in line})
        counting.append(row)
deneme = pd.DataFrame(counting)
deneme.head() # ss al burayı 



,"iros, iros2020, robotics, slam","decentralized, golang, imageboard, ipfs, peer-to-peer","hook, hooks, javascript, react, react-hooks, react-stepper, react-use, react-use-stepper, react-use-wizard, react-wizard, reactjs, stepper, typescript","moses, moses-toolkit, style-transfer, supervised-experiments","auto-scaling, kubernetes, monitoring, prometheus, shard, thanos","tailwindcss, tailwindcss2, typescript, vue3-tailwind2, vuejs, vuejs3","bert, deep-learning, longer-sequences, nlp, transformer","fatfs, filesystem-libraries, lwext4, nintendo-switch, ntfs-3g, usb-msc","catalyst, ios, macos, xamarin","event, handler, hotkey, keyboard, shortcut",...,"firebase, flutter, flutter-apps, flutter-examples, social, social-media, social-network, socialmedia, ui, uikit","battery, conky, cpu, filesystem, gtk, gtk-rs, gtk3, mem, opacity, randy, rust, rust-lang, screenshot","attention, cnn, deep-learning, deeplearning-nlp-models, embeddings, gpt, hugging-face, nlp, nlp-papers, pytorch, torchtext, transformer, tutorials, word2vec","data-visualization, database, database-gui, dummy-data-generator, kpis, sql","keras, telegram-bot, tensorflow, tensorgram","2020, bdci, ccf, ner","image-matting, portrait-matting, video-matting","dart, dart-package, mock, mocking, mockito, mocks, testing","bug-bounty, bugbounty, bugbounty-writeups, facebook-security","alibaba-serverless, aws-serverless, azure-serverless, baidu-serverless, google-serverless, huawei-serverless, serverless, serverless-devs, serverless-devs-tool, serverless-framework, serverless-tool, tencent-serverless"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# 1. Tüm satır toplamlarının toplamının toplam öğe sayısını bulun
tot_item_count = sum(deneme.sum()) #540
#print(tot_item_count)

# 2. İlk 20 öğeyi almak için satırları toplayın ve sıralama azalan düzendedir
item_sum = deneme.sum().sort_values(ascending=False).reset_index().head(n=20)
item_sum.rename(columns={item_sum.columns[0]:'Item_name',item_sum.columns[1]:'Item_count'}, inplace=True)
#print(item_sum)

# 3. Ne kadar katkıda bulunduğunu bilmemiz için yüzdeyi ekleyin.
# X'in toplam yüzdesi, toplam yüzdede x ve üzeri öğelerin yüzdesini, yani kümülatif toplamı belirler.
item_sum['Item_percent'] = item_sum['Item_count']/tot_item_count
item_sum['Tot_percent'] = item_sum.Item_percent.cumsum()
item_sum.head(20) # Yüzdelerle birlikte ilk 20 öğe listesi


# sık kullanılan ögelerin cizimi
obj = (list(item_sum['Item_name'].head(n=20)))
y_pos = np.arange(len(obj))
performans = list(item_sum['Item_count'].head(n=20))

print(performans)

plt.bar(y_pos,performans,align='center',alpha=0.5)
plt.xticks(y_pos,obj,rotation='horizontal')
plt.ylabel('Item_count')
plt.title('Analiz Sonucum')

[5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Text(0.5, 1.0, 'Analiz Sonucum')

In [40]:
print("deneme: ")
print(deneme.shape)


deneme: 
(1448, 2820)


In [54]:
#Dikkate alınacak öğe için Minimum Toplam Satış Yüzdesi.
# transaction islem Dikkate alınacak minimum işlem uzunluğu (yani arka arkaya minimum öğe sayısı).

def prune_dataset(olddf,len_transaction, tot_sales_percent):
    if 'tot_items' in olddf.columns:
        del(olddf['tot_items'])

    # Her öğe için item_count ve toplam öğe sayısını bulma.
    # 3.adım gibi
    Item_count = olddf.sum().sort_values(ascending=False).reset_index()
    tot_items = sum(olddf.sum().sort_values(ascending=False))
    Item_count.rename(columns={Item_count.columns[0]:'Item_name', Item_count.columns[1]:'Item_count'},inplace = True)

    # Öge yuzdesi ve toplam yuzdeyi bulmak icin 3 adıma benzer
    Item_count['Item_percent'] = Item_count['Item_count'] / tot_items
    Item_count['Tot_percent'] = Item_count.Item_percent.cumsum()

    # Toplam yüzde için koşul / minimum eşiğe uyan öğeleri almak.
    selected_items = list(Item_count[Item_count.Tot_percent < tot_sales_percent].Item_name)
    olddf['tot_items'] = olddf[selected_items].sum(axis=1)

    # İşlemin uzunluğu veya bir satırdaki öğe sayısı için koşul / minimum eşiğe uyan öğeleri almak.
    olddf = olddf[olddf.tot_items >= len_transaction]
    del(olddf['tot_items'])

    # Temizlenmis / Kısaltılmıs veri seti

    return olddf[selected_items],Item_count[Item_count.Tot_percent < tot_sales_percent]


In [42]:
#Apriori için uygun bir veri seti elde etmek üzere şimdi len_transaction ve tot_sales_percent için farklı değerler gireceğiz
#Deneme 1
pruneddf, Item_count = prune_dataset(deneme,4,0.6)
print(pruneddf.shape)
print(list(pruneddf.columns))
# Çıktı (Sütun listesi aslında apriori için dikkate aldığımız öğelerdir.)

(0, 1084)
['', 'hacktoberfest', 'hacktoberfest, hacktoberfest2020', 'AutoGL', 'woftool', 'blog, cloudbase, comment, leancloud, mongodb, postgresql, sqlite, valine, vercel', 'Deep-Exemplar-based-Video-Colorization', 'amiga, raspberry', 'obsidian-md, obsidian-plugin', 'Bundler', 'Plasma', 'colorization, computer-vision, deep-learning, gan, generative-adversarial-network, image-generation, image-processing, pytorch', 'activity, android, bundle, bundler, fragment, intent, kotlin, kotlin-extensions, skydoves', 'xrayku', 'awesome-scriptable', 'cli', 'effects.js', 'hamibot', 'android, automation', 'engine', 'bbs, community, developers, forum, java, markdown, sns', 'bert-model, causal-inference, causal-models, causal-spans, causality, conversations, dataset, dialogue-generation, dialogue-systems, emotion, emotion-cause, emotion-cause-pair-extraction, emotion-recognition, emotion-recognition-in-conversation, emotion-tasks, inference, natural-language-inference, reasoning, roberta', 'PanelStudio

In [44]:
# DENEME 2
pruneddf,Item_count = prune_dataset(deneme,4,0.5)
print(pruneddf.shape)
print(list(pruneddf.columns))


(0, 650)
['', 'hacktoberfest', 'hacktoberfest, hacktoberfest2020', 'AutoGL', 'woftool', 'blog, cloudbase, comment, leancloud, mongodb, postgresql, sqlite, valine, vercel', 'Deep-Exemplar-based-Video-Colorization', 'amiga, raspberry', 'obsidian-md, obsidian-plugin', 'Bundler', 'Plasma', 'colorization, computer-vision, deep-learning, gan, generative-adversarial-network, image-generation, image-processing, pytorch', 'activity, android, bundle, bundler, fragment, intent, kotlin, kotlin-extensions, skydoves', 'xrayku', 'awesome-scriptable', 'cli', 'effects.js', 'hamibot', 'android, automation', 'engine', 'bbs, community, developers, forum, java, markdown, sns', 'bert-model, causal-inference, causal-models, causal-spans, causality, conversations, dataset, dialogue-generation, dialogue-systems, emotion, emotion-cause, emotion-cause-pair-extraction, emotion-recognition, emotion-recognition-in-conversation, emotion-tasks, inference, natural-language-inference, reasoning, roberta', 'PanelStudio'

In [46]:
# deneme 3
pruneddf, Item_count = prune_dataset(deneme,2,0.5)
print(pruneddf.shape)
print(list(pruneddf.columns))


(608, 650)
['', 'hacktoberfest', 'hacktoberfest, hacktoberfest2020', 'AutoGL', 'woftool', 'blog, cloudbase, comment, leancloud, mongodb, postgresql, sqlite, valine, vercel', 'Deep-Exemplar-based-Video-Colorization', 'amiga, raspberry', 'obsidian-md, obsidian-plugin', 'Bundler', 'Plasma', 'colorization, computer-vision, deep-learning, gan, generative-adversarial-network, image-generation, image-processing, pytorch', 'activity, android, bundle, bundler, fragment, intent, kotlin, kotlin-extensions, skydoves', 'xrayku', 'awesome-scriptable', 'cli', 'effects.js', 'hamibot', 'android, automation', 'engine', 'bbs, community, developers, forum, java, markdown, sns', 'bert-model, causal-inference, causal-models, causal-spans, causality, conversations, dataset, dialogue-generation, dialogue-systems, emotion, emotion-cause, emotion-cause-pair-extraction, emotion-recognition, emotion-recognition-in-conversation, emotion-tasks, inference, natural-language-inference, reasoning, roberta', 'PanelStudi

In [55]:
# İlk olarak, veri çerçevemizi, orijinal veri kümemiz gibi görünecek, ancak boyutu küçültülmüş bir csv dosyasına dönüştürmemiz gerekir. ½½
# 1'leri uygun öğe adına dönüştürme (sütun adı)

y = list(pruneddf.columns)
print("y",y)
for s in y:
    pruneddf.loc[(pruneddf[s] == 1),s] = s
print(pruneddf)
# Sıfırları Sil
lol = pruneddf.values.tolist()
print(lol)
print(lol)
for a in lol:
    while(0 in a):
        a.remove(0)

# Yeni bir temizlnemiş veri kümesi csv dosyası oluşturma
with open("pruned.csv","w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(lol)

y ['', 'hacktoberfest', 'hacktoberfest, hacktoberfest2020', 'AutoGL', 'woftool', 'blog, cloudbase, comment, leancloud, mongodb, postgresql, sqlite, valine, vercel', 'Deep-Exemplar-based-Video-Colorization', 'amiga, raspberry', 'obsidian-md, obsidian-plugin', 'Bundler', 'Plasma', 'colorization, computer-vision, deep-learning, gan, generative-adversarial-network, image-generation, image-processing, pytorch', 'activity, android, bundle, bundler, fragment, intent, kotlin, kotlin-extensions, skydoves', 'xrayku', 'awesome-scriptable', 'cli', 'effects.js', 'hamibot', 'android, automation', 'engine', 'bbs, community, developers, forum, java, markdown, sns', 'bert-model, causal-inference, causal-models, causal-spans, causality, conversations, dataset, dialogue-generation, dialogue-systems, emotion, emotion-cause, emotion-cause-pair-extraction, emotion-recognition, emotion-recognition-in-conversation, emotion-tasks, inference, natural-language-inference, reasoning, roberta', 'PanelStudio', 'RECC

In [56]:
# APRİORİ
import csv
import itertools

dosya_adi = open("pruned.csv","r")
#Minimum Destek Sayısı
min_support = 0.04
kurallar = "Kurallar.txt"
freqItemsets = "kuralFrekans.txt"
#Minimum Güven
min_confidence = 0.30

# Sık kullanılan ögelerin frekansını bulma
def OneItemSets():
    # Liste öğelerindeki tüm 1 öğe kümelerini ve bunların dict sayılarındaki sayısı
    DataCaptured = csv.reader(dosya_adi,delimiter = ',')
    data = list(DataCaptured)
    for e in data:
        e = sorted(e)
    count = {}
    for items in data:
        for item in items:
            if item not in count:
                count[(item)] = 1
            else:
                count[(item)] = count[(item)] + 1

    count2 = {k: v for k, v in count.items() if v >= min_support*1446}

    return  count2,data


#Ck, Lk'nin bir üst kümesidir. Prune Step'in bir parçasıdır. üyeleri sık olabilir veya olmayabilir, ancak tüm sık kullanılan k öğeleri Ck'ye dahil edilmiştir.
# İki Lk'ye katılarak oluşturuldu

def generateCk(LK_1,flag,data):
    Ck = []
    if flag == 1:
        flag = 0
        for item1 in LK_1:
            for item2 in LK_1:
                if item2 > item1:
                    Ck.append((item1,item2))
        print("C2: ",Ck[1:3])
        print("length: ", len(Ck))
        print()

    else:
        for item in LK_1:
            k = len(item)
        for item1 in LK_1:
            for item2 in LK_1:
                if (item1[:-1] == item2[:-1]) and (item1[-1] != item2[-1]):
                    if item1[-1] > item2[-1]:
                        Ck.append(item2 + (item1[-1],))
                    else:
                        Ck.append(item1 + (item2[-1],))
        print("C" + str(k+1) + ": ", Ck[1:3])
        print("Length : ", len(Ck))
        print()

    L = generateLk(set(Ck),data)
    return  L,flag
# Ck'deki öğe bir işleme aitse, onu listeye alır Ct Sonra Ct, L oluşturacak şekilde eşlenir
# K sık tag kümesi için
def generateLk(Ck, data):

    count = {}
    for itemset in Ck:
        #print(itemset)
        for transaction in data:
            if all(e in transaction for e in itemset):
                if itemset not in count:
                    count[itemset] = 1
                else:
                    count[itemset] = count[itemset] + 1

    print("Ct Length : ", len(count))
    print()

    count2 = {k: v for k, v in count.items() if v >= min_support*1446}
    print("L Length : ", len(count2))
    print()
    return count2

# Sık kullanılan öğe kümelerinden ilişkilendirme kuralları oluşturur
def rulegenerator(fitems):
    counter = 0
    for itemset in fitems.keys():
        if isinstance(itemset, str):
            continue
        length = len(itemset)

        union_support = fitems[tuple(itemset)]
        for i in range(1, length):

            lefts = map(list, itertools.combinations(itemset, i))
            for left in lefts:
                if len(left) == 1:
                    if ''.join(left) in fitems:
                        leftcount = fitems[''.join(left)]
                        conf = union_support / leftcount
                else:
                    if tuple(left) in fitems:
                        leftcount = fitems[tuple(left)]
                        conf = union_support / leftcount
                if conf >= min_confidence:
                    fo = open(kurallar, "a+")
                    right = list(itemset[:])
                    for e in left:
                        right.remove(e)
                    fo.write(str(left) + ' (' + str(leftcount) + ')' + ' -> ' + str(right) + ' (' + str(fitems[''.join(right)]) + ')' + ' [' + str(conf) + ']' + '\n')
                    print(str(left) + ' -> ' + str(right) + ' (' + str(conf) + ')')
                    counter = counter + 1
                    #Greater than 1???
                    fo.close()
    print(counter, "rules generated")

def plotitemfreq(L):
    aux = [(L[key], key) for key in L]
    aux.sort()
    aux.reverse()
    z = aux[0:20]
    print(z)
    df = pd.DataFrame(z, columns = ['Count', 'Word'])
    df['Count']=pd.to_numeric(df['Count'])
    print(df.info())
    df.plot.bar(x='Word', y='Count')

def apriori():
    L, data = OneItemSets()
    flag = 1
    FreqItems = dict(L)
    while(len(L) != 0):
        fo = open(freqItemsets, "a+")
        for k, v in L.items():

            fo.write(str(k) + ' >>> ' + str(v) + '\n\n')
        fo.close()
        plotitemfreq(L)

        L, flag = generateCk(L, flag, data)
        FreqItems.update(L)
    rulegenerator(FreqItems)


apriori()

#http://deryagunduz.com/?p=1013

[(608, '')]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Count   1 non-null      int64 
 1   Word    1 non-null      object
dtypes: int64(1), object(1)
memory usage: 144.0+ bytes
None
C2:  []
length:  0

Ct Length :  0

L Length :  0

0 rules generated
